In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Dense, Concatenate, BatchNormalization

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
csv_file_path = '/content/final_data.csv'
data = pd.read_csv(csv_file_path)

# Split the data into a 60:40 ratio
train_data, test_data = train_test_split(data, test_size=0.5, random_state=42)

# Save the split data into new CSV files
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the testing data CSV file into a DataFrame
test_data = pd.read_csv('test_data.csv')

# Split the testing data into labeled (50%) and unlabeled (50%) parts
labeled_data, unlabeled_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Save the labeled and unlabeled data into separate CSV files
labeled_data.to_csv('labeled_data.csv', index=False)
unlabeled_data.drop(columns=['Label'], inplace=True)  # Drop the label column
unlabeled_data.to_csv('unlabeled_data.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the testing data CSV file into a DataFrame
test_data = pd.read_csv('test_data.csv')

# Split the testing data into labeled (50%) and unlabeled (50%) parts
labeled_data, unlabeled_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Save the labeled and unlabeled data into separate CSV files
labeled_data.to_csv('labeled_data.csv', index=False)
unlabeled_data.drop(columns=['Label'], inplace=True)  # Drop the label column
unlabeled_data.to_csv('unlabeled_data.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re

# Load data
data = pd.read_csv('/content/train_data.csv')

# Extract URL strings and labels
X = data[['URL','Content-Type','Payload']]
y = data['Label']


X = X.astype(str)
print(type(X))
print(len(X))
Y = y.astype(str)
print(type(Y))
print(len(Y))


<class 'pandas.core.frame.DataFrame'>
1315
<class 'pandas.core.series.Series'>
1315


In [ ]:
special_characters = ["'", '"', '<', '>', '+', '-', '_', '*', '=', '{', '}', '(', ')', '[', ']', '∼', '/',
                      '\\', '#', ':', ';', '?', '!', '&', '@']

tokenized_payloads = []

def tokenize_and_segment(X):
    for payload in X:
        segmented_payload = []
        current_segment = ""

        for char in payload:
            if char in special_characters:
                if current_segment:
                    segmented_payload.append(current_segment)
                current_segment = ""
                segmented_payload.append(char)
            else:
                current_segment += char

        if current_segment:
            segmented_payload.append(current_segment)

        tokenized_payloads.append(segmented_payload)

    return tokenized_payloads

tokenized_payloads = tokenize_and_segment(X['Payload'])

In [ ]:
vocabulary = {}
index = 0
for sample in tokenized_payloads:
    for token in sample:
        if token not in vocabulary:
            vocabulary[token] = index
            index += 1

In [ ]:
# Initialize a Tokenizer
tokenizer = Tokenizer(filters=special_characters, oov_token='<OOV>')

# Fit the tokenizer on payload data to build the vocabulary
tokenizer.fit_on_texts(X['Payload'])

# Convert text to sequences of indices
sequences = tokenizer.texts_to_sequences(X['Payload'])

In [ ]:
# Step 4: Sequence Padding/Truncation
max_sequence_length = 498 # Adjust this value based on your requirements
#Convert tokens to indices based on vocabulary
sequences = [[vocabulary[token] for token in sample] for sample in tokenized_payloads]
# Pad sequences to a maximum length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating = 'post')

# Print the padded sequences
print(len(sequences))
print(len(padded_sequences))


1315
1315


In [ ]:
# Vocabulary creation
vocab_size = len(vocabulary)
print(vocab_size)


1602


In [ ]:
# Label encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)
print(len(encoded_labels))

59658


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd


# Vocabulary creation
vocabulary_size = len(vocabulary)
embedding_dim = 100  # Adjust this based on your requirements
max_sequence_length = 498  # Adjust this based on your requirements
num_classes = len(label_encoder.classes_)

# Step 5: Embedding Layer
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim)(input_layer)

# Step 6: Convolutional Layer
num_filters = 128  # Adjust this based on your requirements
filter_sizes = [3, 4, 5]  # You can experiment with different filter sizes
convolutional_layers = []

for filter_size in filter_sizes:
    conv_layer = Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')(embedding_layer)
    pool_layer = MaxPooling1D(pool_size=max_sequence_length - filter_size + 1)(conv_layer)
    convolutional_layers.append(pool_layer)

merged_layer = tf.keras.layers.concatenate(convolutional_layers, axis=1)
flatten_layer = Flatten()(merged_layer)

# Step 7: Fully Connected Layers
hidden_units = 128  # Adjust this based on your requirements
dense_layer = Dense(units=hidden_units, activation='relu')(flatten_layer)
output_layer = Dense(units=num_classes, activation='softmax')(dense_layer)

# Create the model
text_cnn_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
text_cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
text_cnn_model.summary()

# Train the model
batch_size = 64  # Adjust this based on your resources
num_epochs = 10  # Adjust this based on your requirements

text_cnn_model.fit(padded_sequences, encoded_labels, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 498)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 498, 100)     4069600     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 496, 128)     38528       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 495, 128)     51328       ['embedding[0][0]']              
                                                                                              

In [ ]:
from keras.models import load_model

In [ ]:
text_cnn_model.save("text_CNNacc72.h5")
# Load the pre-trained TextCNN model
pretrained_model = load_model("text_CNNacc72.h5")

# Remove the last layer (output layer) from the pre-trained model
#pretrained_model.layers.pop()


text_cnn with batch normalization

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load and preprocess your dataset

# Define vocabulary, sequences, and other parameters

# Build the TextCNN model with Batch Normalization
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim)(input_layer)

# Add Convolutional and BatchNormalization layers
convolutional_layers = []

for filter_size in filter_sizes:
    conv_layer = Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')(embedding_layer)
    batch_norm_layer = BatchNormalization()(conv_layer)  # Adding Batch Normalization
    pool_layer = MaxPooling1D(pool_size=max_sequence_length - filter_size + 1)(batch_norm_layer)
    convolutional_layers.append(pool_layer)

merged_layer = tf.keras.layers.concatenate(convolutional_layers, axis=1)
flatten_layer = Flatten()(merged_layer)

# Add BatchNormalization to the Dense layers
dense_layer = Dense(units=hidden_units, activation='relu')(flatten_layer)
batch_norm_dense = BatchNormalization()(dense_layer)  # Adding Batch Normalization
output_layer = Dense(units=num_classes, activation='softmax')(batch_norm_dense)

# Create the model
text_cnn_model_with_bn = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
text_cnn_model_with_bn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 64
num_epochs = 10

text_cnn_model_with_bn.fit(padded_sequences, encoded_labels, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)

data Perturbation

In [ ]:
import numpy as np
import random

def apply_keyword_avoidance_perturbation(original_sample, keyword_library):
    def randomize_number(n):
        return str(np.random.randint(10))

    def randomize_letter(c):
        if c.isalpha():
            return chr(np.random.randint(ord('a'), ord('z') + 1))
        return c

    perturbed_words = []
    for word in original_sample:
        if word in keyword_library:
            perturbed_words.append(word)
        else:
            special_characters = ['\'', '"', '<', '>', '+', '-', '_', '*', '=', '{', '}', '(', ')', '[', ']', '∼', '/',
                                  '\\', '#', ':', ';', '?', '!', '&', '@', '%']
            if any(char in special_characters for char in word):
                perturbed_words.append(word)
            else:
                perturbed_word = ''.join(randomize_number(c) if c.isdigit() else randomize_letter(c) for c in word)
                perturbed_words.append(perturbed_word)

    return perturbed_words

# Define the keyword library
keyword_library = {"login", "password", "malicious", "attack", "exploit", "hacker", "virus",
                   "SELECT", "UNION", "AND", "http", "script"}

# Example usage of data augmentation with payload
original_payload = tokenized_payloads[0]  # Use any payload from your tokenized_payloads list

augmented_payload = apply_keyword_avoidance_perturbation(original_payload, keyword_library)

print("Original Payload:", original_payload)
print("Augmented Payload:", augmented_payload)


Original Payload: ['nan']
Augmented Payload: ['npy']


In [ ]:
import numpy as np
import random

def apply_keyword_avoidance_perturbation(original_url, keyword_library):
    def randomize_number(n):
        return str(np.random.randint(10))

    def randomize_letter(c):
        if c.isalpha():
            return chr(np.random.randint(ord('a'), ord('z') + 1))
        return c

    # Split the original URL into components using "/", "=", and "."
    components = re.split(r'([/=\.&?])', original_url)
    perturbed_components = []

    for i, component in enumerate(components):
        if component in ('/', '=', '.',"&","?"):
            perturbed_components.append(component)
            continue

        if any(keyword in component for keyword in keyword_library):
            perturbed_components.append(component)
        else:
            perturbed_component = ''.join(randomize_number(c) if c.isdigit() else randomize_letter(c) for c in component)
            perturbed_components.append(perturbed_component)

    # Join the perturbed components back into the URL using original separators
    perturbed_url = ''.join(perturbed_components)
    return perturbed_url

# Define the keyword library
keyword_library = {"login","user", "password", "malicious", "attack", "exploit", "hacker", "virus",
                   "com", "co", "cn", "in", "http","php"}

import re  # Import the re module for regular expressions

# Example usage of data augmentation with URL
original_url = "http://example.com/login.php?user=admin&password=123456"

augmented_url = apply_keyword_avoidance_perturbation(original_url, keyword_library)

print("Original URL:", original_url)
print("Augmented URL:", augmented_url)


Original URL: http://example.com/login.php?user=admin&password=123456
Augmented URL: http://zupinsl.com/login.php?user=admin&password=853194
